Libraries

In [ ]:
import os
import pandas as pd
import xarray as xr
import numpy as np
import pyvista as pv
from scipy.interpolate import griddata

Global *pyvista* settings

In [ ]:
pv.set_jupyter_backend('pythreejs')

pv.global_theme.background = 'white'
pv.global_theme.window_size = [400, 400]
pv.global_theme.antialiasing = True
# pv.global_theme.show_edges = True
# pv.global_theme.edge_color = 'white'
# pv.global_theme -> visualize all the theme options

Constants

In [ ]:
CTE = 111.0 # constant to convert lat. and lon. values to km
SCALE = 100 # number to scale depth

# Load Slab2 model
Check if there is a directory with the Slab2.0 model

In [ ]:
pwd = os.path.join("src")
if not os.path.exists(pwd):
    print('Error: Could not find source directory to Slab2.0.')

dep_file = os.path.join(pwd,'sam_slab2_dep_02.23.18.xyz')
thk_file = os.path.join(pwd,'sam_slab2_thk_02.23.18.xyz')
str_file = os.path.join(pwd,'sam_slab2_str_02.23.18.xyz')
dip_file = os.path.join(pwd,'sam_slab2_dip_02.23.18.xyz')

Read the top surface depth of the slab

In [ ]:
data = pd.read_csv(dep_file, names=['lon', 'lat', 'dep'])
dep = data.pivot_table(index='lon', columns='lat', values='dep', dropna=False).T.values
dep = xr.DataArray(1.0E3 * dep, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
dep.attrs['units'] = 'm'
dep = dep.to_dataset(name='dep')

Read the thickness of the slab

In [ ]:
data = pd.read_csv(thk_file, names=['lon', 'lat', 'thk'])
thk = data.pivot_table(index='lon', columns='lat', values='thk', dropna=False).T.values
thk = xr.DataArray(1.0E3 * thk, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
thk.attrs['units'] = 'm'
thk = thk.to_dataset(name='thk')

Read the strike angle of the slab

In [ ]:
data = pd.read_csv(str_file, names=['lon', 'lat', 'str'])
strike = data.pivot_table(index='lon', columns='lat', values='str', dropna=False).T.values
strike *= np.pi / 180.
strike = xr.DataArray(strike, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
strike.attrs['units'] = 'rad'
strike = strike.to_dataset(name='str')

Read the dip angle of the slab

In [ ]:
data = pd.read_csv(dip_file, names=['lon', 'lat', 'dip'])
dip = data.pivot_table(index='lon', columns='lat', values='dip', dropna=False).T.values
dip *= np.pi / 180.
dip = xr.DataArray(dip, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
dip.attrs['units'] = 'rad'
dip = dip.to_dataset(name='dip')

Merge all the DataArrays into a Dataset

In [ ]:
slab2 = xr.merge([dep, thk, strike, dip])

# Make Slab2 bottom surface and *pyvista* meshes
Make *pyvista* mesh

In [ ]:
x = slab2.coords['lon']
y = slab2.coords['lat']
x, y = np.meshgrid(x, y)
z = slab2.dep.values / 1_000.0 / SCALE

top_mesh = pv.StructuredGrid(x, y, z)

top_mesh["top"] = (slab2.dep.values).flatten(order="F")
top_mesh["dip"] = (slab2.dip.values).flatten(order="F")
top_mesh["str"] = (slab2.str.values).flatten(order="F")

In [ ]:
bot_x = slab2.coords["lon"] * CTE
bot_y = slab2.coords["lat"] * CTE
bot_x, bot_y = np.meshgrid(bot_x, bot_y)

# Calculate dip correction
x_corr_dip = np.sin(slab2.dip.values) * slab2.thk.values / 1_000
z_corr_dip = np.cos(slab2.dip.values) * slab2.thk.values / 1_000

# Calculate strike correction
x_corr_str = np.sin(slab2.str.values) * x_corr_dip / 1_000
y_corr_str = np.cos(slab2.str.values) * x_corr_dip / 1_000

# Apply correction to bot_z
bot_z = ((slab2.dep.values / 1_000)  - z_corr_dip) / SCALE

# Ref. bot mesh (will be removed!)
ref_mesh = pv.StructuredGrid(x, y, bot_z)
ref_mesh["bot"] = bot_z.flatten(order="F")
ref_mesh["dip"] = (slab2.dip.values).flatten(order="F")

# Apply corrections to bot_x and bot_y
for i in range(bot_z.shape[0]):
    for j in range(bot_z.shape[1]):
        if (~np.isnan(bot_z[i,j])):
            bot_x[i,j] = bot_x[i,j] - x_corr_dip[i,j] + x_corr_str[i,j]
            bot_y[i,j] = bot_y[i,j] - y_corr_str[i,j]

# Filter np.nan and interpolate Slab2 bottom surface
points = np.zeros((bot_z.size, 3))
aux = 0
for j in range(bot_z.shape[1]):
    for i in range(bot_z.shape[0]):
        if (~np.isnan(bot_z[i,j])):
            points[aux,0] = bot_x[i,j] / CTE
            points[aux,1] = bot_y[i,j] / CTE
            points[aux,2] = bot_z[i,j]
            aux += 1
values = points[:aux,2]            
points = points[:aux,:2]
interp_z = griddata(points, values, (x.T, y.T), method='cubic')

# Make grid
bot_mesh = pv.StructuredGrid(x, y, interp_z.T)
bot_mesh["bot"] = (interp_z.T).flatten(order="F")

In [ ]:
points = bot_mesh.glyph(scale=True, factor=0.1)

In [ ]:
plotter = pv.Plotter(multi_samples=4)
plotter.add_mesh(top_mesh, scalars="top")
plotter.add_mesh(bot_mesh, scalars="bot")
# plotter.add_mesh(points, color='black')
# plotter.add_mesh(ref_mesh, scalars="dip", cmap="seismic")
plotter.add_bounding_box()
plotter.camera_position = 'xy'
plotter.show()